In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

DECIMALS = 10**18
NULL_ADDRESS = '0x0000000000000000000000000000000000000000'

balances = pd.read_csv('./stake/contract_balances.csv')
balances.total_supply = balances.total_supply.map(int)
balances.qty = balances.qty.map(int)

eth_txs = pd.read_csv('./stake/ethereum_token_transfers.csv')
eth_txs.value = eth_txs.value.map(int)
eth_txs = eth_txs.drop_duplicates()

ids = pd.read_csv('identities.csv')

ValueError: invalid literal for int() with base 10: 'total_supply'

In [ ]:
# Who contributed to STAKE's private sale and how much do they have left?

# Find the private sale contract (psc) address
psc_addr = ids[ids['name'].str.contains("stake private sale")].reset_index(drop=True).loc[0, 'address']

# Find all txns that the psc_addr sent to and calculate sum
bought = eth_txs[eth_txs['from'] == psc_addr].groupby('to').value.agg(['sum'])
bought.rename(columns={'sum': 'bought'}, inplace=True)

# Get all advisor addresses
addresses = list(bought.index)

# Find all txns that the private investors sent to and calculate sum
sold = eth_txs[eth_txs['from'].isin(addresses)].groupby('from').value.agg(['sum'])
sold.rename(columns={'sum': 'sold'}, inplace=True)

# Join the 2 dataframes together and calculate remaining
df = bought.join(sold)

# If sold is > bought, it means that this address received stake from outside
df['sold'] = np.where(df.bought < df.sold, df.bought, df.sold)
df['remaining'] = df.bought - df.sold

df = df.stack().reset_index(name='qty')
df.rename(columns={'to': 'address', 'level_1': 'type'}, inplace=True)
df.qty = df.qty.map(int) / DECIMALS

# Plot stacked bar chart
fig = px.bar(df[df.type != 'bought'], x="address", y="qty",
             color='type', title="Stake private sale investors' balances")
fig.update_traces(texttemplate = "%{value:.2f}")
fig.show()

In [ ]:
# Stake distribution to private sale investors over time

# Stake going out of the private sale contract
outof = eth_txs[eth_txs['from'] == psc_addr].copy().reset_index(drop=True)
outof.loc[-1] = {'from': '', 'to': '', 'value': 0, 'block': outof.loc[0, 'block'] - 1, 'tx_hash': ''}
outof.sort_index(inplace=True)
outof['balance'] = 1970951
outof.value = outof.value.map(int) / DECIMALS

for i in range(1, len(outof)):
    outof.iat[i, 5] = outof.iat[i-1, 5] - outof.iat[i, 2]
    

fig = px.line(x=outof.block, y=outof.balance, labels={'x':'Block number', 'y':'Stake balance'},
              title='Stake balance over time in 0x3cFE51b61E25750ab1426b0072e5D0cc5C30aAfA')
fig.update_xaxes(tickformat = ".0f")
fig.update_yaxes(rangemode = 'tozero')
fig.show()

In [ ]:
# Who are STAKE advisor's and how much do they have left?

# Find the advisor contract (ac) address
ac_addr = ids[ids['name'].str.contains("stake advisor")].reset_index(drop=True).loc[0, 'address']

# Find all txns that the ac_addr sent to and calculate sum
bought = eth_txs[eth_txs['from'] == ac_addr].groupby('to').value.agg(['sum'])
bought.rename(columns={'sum': 'bought'}, inplace=True)

# Get all advisor addresses
addresses = list(bought.index)

# Find all txns that the advisors sent to and calculate sum
sold = eth_txs[eth_txs['from'].isin(addresses)].groupby('from').value.agg(['sum'])
sold.rename(columns={'sum': 'sold'}, inplace=True)

# Join the 2 dataframes together and calculate remaining
df = bought.join(sold)

# If sold is > bought, it means that this address received stake from outside
df['sold'] = np.where(df.bought < df.sold, df.bought, df.sold)
df['remaining'] = df.bought - df.sold

df = df.stack().reset_index(name='qty')
df.rename(columns={'to': 'address', 'level_1': 'type'}, inplace=True)
df.qty = df.qty.map(int) / DECIMALS

# Plot stacked bar chart
fig = px.bar(df[df.type != 'bought'], x="address", y="qty",
             color='type', title="Stake advisors' balances")
fig.update_traces(texttemplate = "%{value:.2f}")
fig.show()

In [ ]:
# Stake distribution to advisors over time

# Stake going out of the advisor distribution contract
outof = eth_txs[eth_txs['from'] == ac_addr].copy().reset_index(drop=True)
outof.loc[-1] = {'from': '', 'to': '', 'value': 0, 'block': outof.loc[0, 'block'] - 1, 'tx_hash': ''}
outof.sort_index(inplace=True)
outof['balance'] = 651000
outof.value = outof.value.map(int) / DECIMALS

for i in range(1, len(outof)):
    outof.iat[i, 5] = outof.iat[i-1, 5] - outof.iat[i, 2]
    

fig = px.line(x=outof.block, y=outof.balance, labels={'x':'Block number', 'y':'Stake balance'},
              title='Stake balance over time in 0x0218B706898d234b85d2494DF21eB0677EaEa918')
fig.update_xaxes(tickformat = ".0f")
fig.update_yaxes(rangemode = "tozero")
fig.show()

In [ ]:
# Who are your top LPs on balancer?
b_txs = pd.read_csv('./stake/ethereum_balancer_transfers.csv')
b_txs.value = b_txs.value.map(int)
b_txs = b_txs.drop_duplicates()

# Who received balancer LP tokens
into = b_txs.groupby('to').value.agg('sum')

# Who sent out balancer LP tokens
outof = b_txs.groupby('from').value.agg('sum')
outof = -outof

# Get LP token balance in each address
b_balances = pd.concat([into, outof])
balancer_df = b_balances.groupby(b_balances.index).agg('sum').sort_values(ascending=False)
balancer_df = balancer_df.drop(NULL_ADDRESS).reset_index()
balancer_df.rename(columns={'index': 'address'}, inplace=True)

balances[(balances.who == 'balancer') &
         (balances.blockchain == 'ethereum')].reset_index(drop=True).iloc[0, 3]

# convert balancer LP tokens into stake holdings
temp = balances[(balances.who == 'balancer') &
         (balances.blockchain == 'ethereum')].reset_index(drop=True)
stake_on_balancer = temp.iloc[0, 3]
total_supply_lpt = temp.iloc[0, 4]
balancer_df.value = balancer_df.value * stake_on_balancer / total_supply_lpt / DECIMALS

# value > 0 = current liquidity providers
# value == 0 = ex liquidity providers
# value < 0 = should only have 1 which is the null address (already dropped it a few lines above)

# Show the top 5 LPs, sum the remaining
df = balancer_df.copy()
tail = df.loc[5:]
everyone_else_balance = tail.value.sum()
df = df.head(5)
df.loc[5] = ['everyone else', everyone_else_balance]

fig = px.pie(df, values='value', names='address', title='Top LPs on balancer')
fig.update_traces(texttemplate = "%{value:.2f} stake (%{percent})")
fig.show()

In [ ]:
# Who are your top LPs on uniswap?
u_txs = pd.read_csv('./stake/ethereum_uniswap_transfers.csv')
u_txs.value = u_txs['value'].map(int)
u_txs = u_txs.drop_duplicates()

# Who received uniswap LP tokens
into = u_txs.groupby('to').value.agg('sum')

# Who sent out uniswap LP tokens
outof = u_txs.groupby('from').value.agg('sum')
outof = -outof

# Get LP token balance in each address
u_balances = pd.concat([into, outof])
uniswap_df = u_balances.groupby(u_balances.index).agg('sum').sort_values(ascending=False)
uniswap_df = uniswap_df.drop(NULL_ADDRESS).reset_index()
uniswap_df.rename(columns={'index': 'address'}, inplace=True)

# value > 0 = current liquidity providers
# value == 0 = ex liquidity providers
# value < 0 = should only have 1 which is the null address (already dropped it a few lines above)

# convert uniswap LP tokens into stake holdings
temp = balances[(balances.who == 'uniswap') &
         (balances.blockchain == 'ethereum')].reset_index(drop=True)
stake_on_uniswap = temp.iloc[0, 3]
total_supply_lpt = temp.iloc[0, 4]
uniswap_df.value = uniswap_df.value * stake_on_uniswap / total_supply_lpt / DECIMALS

# Show the top 10 LPs, sum the remaining
df = uniswap_df.copy()
tail = df.loc[10:]
everyone_else_balance = tail.value.sum()
df = df.head(10)
df.loc[10] = ['everyone else', everyone_else_balance]

fig = px.pie(df, values='value', names='address', title='Top LPs on Uniswap')
fig.update_traces(texttemplate = "%{value:.2f} stake (%{percent})")
fig.show()

In [ ]:
# Who are your top stakers on EasyStaking?

# Filter transactions to only look at those that are going in and out of
# EasyStaking's smart contracts. Theory is that whatever goes in must come out.
# If it's not out, means address still has some stake tokens on EasyStaking
df = eth_txs[(eth_txs["from"] == '0xecbCd6D7264e3c9eAc24C7130Ed3cd2B38F5A7AD') |
             (eth_txs["to"] == '0xecbCd6D7264e3c9eAc24C7130Ed3cd2B38F5A7AD')]

into = df.groupby('to').value.agg('sum')
into = -into

outof = df.groupby('from').value.agg('sum')

es_balances = pd.concat([into, outof])

# Any balances < 0 are those that withdrew with profits from staking
# Any balances = 0 are those that withdrew before any profits
# Any balances > 0 are those that still have tokens staked e.g.
# 0x834f3a4D3543Af46DcbE2e4f72520095b673D483 or 0x1D1c6BE4Ea347d1A684A0dF351fedd8AD6771ac0
es_df = es_balances.groupby(es_balances.index).agg('sum').sort_values(ascending=False)

# Drop the null address because it's not a "staker" per se.
es_df = es_df.drop(NULL_ADDRESS).reset_index()
es_df.rename(columns={'index': 'address'}, inplace=True)

# Take only the top 20 that are existing stakers
df = es_df[es_df.value > 0]
tail = df.loc[20:]
everyone_else_balance = tail.value.sum()
df = df.head(20)
df.loc[20] = ['everyone else', everyone_else_balance]
df.value = df.value.map(int) / DECIMALS

fig = px.pie(df, values='value', names='address', title='Top stakers on easy staking')
fig.update_traces(texttemplate = "%{value:.2f} stake (%{percent})")
fig.show()

In [ ]:
# Who are your top stakers on PosDAO?
xdai_txs = pd.read_csv('./stake/xdai_token_transfers.csv')
xdai_txs.value = xdai_txs['value'].map(int)
xdai_txs = xdai_txs.drop_duplicates()
POSDAO_ADDR = '0x2DdB8A7541e6cAA50F74e7FACFF9Fe9da00e0A6c'

# Get stake balances in wallet
into = xdai_txs.groupby('to').value.agg('sum')
outof = xdai_txs.groupby('from').value.agg('sum')
outof = -outof

## Drop the null address
# into = into.drop(NULL_ADDRESS)
# outof = outof.drop(NULL_ADDRESS)

xdai_total_balances = pd.concat([into, outof])

total_df = xdai_total_balances.groupby(xdai_total_balances.index).agg('sum')
total_df = total_df.sort_values(ascending=False).reset_index()
total_df.rename(columns={'index': 'address'}, inplace=True)

total_df['type'] = "Wallet"

# Get transfers to and from the POSDAO address
df = xdai_txs[(xdai_txs["from"] == POSDAO_ADDR) |
              (xdai_txs["to"] == POSDAO_ADDR)]

into = df.groupby('to').value.agg('sum')
outof = df.groupby('from').value.agg('sum')
into = -into

posdao_balances = pd.concat([into, outof])

posdao_df = posdao_balances.groupby(posdao_balances.index).agg('sum').sort_values(ascending=False).reset_index()
posdao_df.rename(columns={'index': 'address'}, inplace=True)
posdao_df['type'] = 'PosDao'

df = pd.concat([total_df, posdao_df], ignore_index=True)
df.value = df.value / DECIMALS

addresses = list(df.groupby('address').value.sum().sort_values(ascending=False).head(20).index)

fig = px.bar(df[df.address.isin(addresses)], x="address", y="value", 
             color='type', title="Total stake balance (wallet + posdao)")
fig.show()

total_df

# df = df.groupby('address').value.agg('sum').sort_values(ascending=False).reset_index()

# tail = df.loc[20:]
# everyone_else_balance = tail.value.sum()
# df = df.head(20)
# df.loc[20] = ['everyone else', everyone_else_balance]
# df.value = df.value.map(int) / DECIMALS

# fig = px.pie(dfz, values='value', names='address', title='Top stakers / delegators on PosDAO')
# fig.update_traces(texttemplate = "%{value:.2f} stake (%{percent})", textposition='outside')
# fig.show()

In [ ]:
# Get total balance breakdown

into = eth_txs.groupby('to').value.agg('sum')
outof = eth_txs.groupby('from').value.agg('sum')

# Drop the null address because it represents a mint event
outof = outof.drop(NULL_ADDRESS)
outof = -outof

total_balances = pd.concat([into, outof])

total_df = total_balances.groupby(total_balances.index).agg('sum').sort_values(ascending=False).reset_index()
total_df.rename(columns={'index': 'address'}, inplace=True)

total_df.value.sum()

In [ ]:
# Get total balance breakdown

into = eth_txs.groupby('to').value.agg('sum')
outof = eth_txs.groupby('from').value.agg('sum')

# Drop the null address because it represents a mint event
outof = outof.drop(NULL_ADDRESS)
outof = -outof

total_balances = pd.concat([into, outof])

total_df = total_balances.groupby(total_balances.index).agg('sum').sort_values(ascending=False).reset_index()
total_df.rename(columns={'index': 'address'}, inplace=True)

# total_df, balancer_df and uniswap_df have no values < 0 since 
# I already dropped them (null address)
# es_df and omni_df have multiple values < 0 because stakers 
# who are rewarded with stake will take out more stake than they put in.
# should we drop the rows with values < 0? 

total_df['type'] = 'Wallet'
balancer_df['type'] = 'Balancer'
uniswap_df['type'] = 'Uniswap'
es_df['type'] = 'EasyStaking'
omni_df['type'] = 'OmniBridge'

df = pd.concat([total_df, balancer_df, uniswap_df,
                es_df[es_df.value > 0], omni_df[omni_df.value > 0]],
                ignore_index=True)

# Gonna just get the top 10 overall otherwise there will be too many addresses lol
df = df.groupby('address').value.agg('sum').sort_values(ascending=False)
# Drop the addresses that are not very relevant
# 0x9BC4a93883C522D3C79c81c2999Aab52E2268d03 = main distribution contract
# 0x86edd0c110D1Fc7F8A5e1108623b3B1B4E3740f9 = ecosystem fund and foundation rewards contract
# 0x1DDF0976Ac842C696d01a86b39d25b067Ed8C7ff = balancer pool contract
# 0x3B3d4EeFDc603b232907a7f3d0Ed1Eea5C62b5f7 = uniswap pool contract
# 0xecbCd6D7264e3c9eAc24C7130Ed3cd2B38F5A7AD = easy staking proxy contract
# 0x00000000219ab540356cBB839Cbe05303d7705Fa = eth 2.0 contract
# 0x88ad09518695c6c3712AC10a214bE5109a655671 = omni bridge

df2 = df.drop(['0x9BC4a93883C522D3C79c81c2999Aab52E2268d03',
               '0x86edd0c110D1Fc7F8A5e1108623b3B1B4E3740f9',
               '0x1DDF0976Ac842C696d01a86b39d25b067Ed8C7ff',
               '0x3B3d4EeFDc603b232907a7f3d0Ed1Eea5C62b5f7',
               '0xecbCd6D7264e3c9eAc24C7130Ed3cd2B38F5A7AD',
               '0x88ad09518695c6c3712AC10a214bE5109a655671',
               '0x00000000219ab540356cBB839Cbe05303d7705Fa',
               NULL_ADDRESS]).reset_index()
# df2 = df.drop(['0x9BC4a93883C522D3C79c81c2999Aab52E2268d03']).reset_index()

df2.rename(columns={'index': 'address'}, inplace=True)

tail = df2.loc[20:]
everyone_else_balance = tail.value.sum()
df2 = df2.head(20)
df2.loc[20] = ['everyone else', everyone_else_balance]
df2.value = df2.value.map(int) / DECIMALS

fig = px.pie(df2, values='value', names='address', title='Top 30 stake holdings across wallets, dexes & staking platforms')
fig.update_traces(texttemplate = "%{value:.2f} stake (%{percent})", textposition='outside')
fig.show()

In [ ]:

# Gonna just get the top 20 overall otherwise there will be too many addresses lol
# addresses = list(df.groupby('address').value.agg('sum').sort_values(ascending=False).head(20).index)

# df = df[df.address.isin(addresses)]

# # Plot stacked bar chart
# fig = px.bar(df, x="address", y="value", 
#              color='type', title="Total stake balance (wallet + dexes + staking platforms)")
# fig.show()


In [ ]:
# How many tokens accumulated between block A and block B

In [ ]:
# # Get largest token balances that are not exchanges,
# # contracts that belong to stake, staking platforms, null address

# exchanges = list(ids[ids['category'].isin(['cex', 'dex'])].reset_index(drop=True).address)
# null = [ids.loc[0,'address']]
# stake_distribution = [ids.loc[1,'address']]
# staking = [
#     '0x88ad09518695c6c3712AC10a214bE5109a655671',
#     '0xecbCd6D7264e3c9eAc24C7130Ed3cd2B38F5A7AD'
# ]

# ignored = exchanges + null + stake_distribution + staking


# # Remove the addresses that are to be ignored & value < 100
# df = df[(df.address.isin(ignored) == False)]

# balances = pd.merge(df, ids, how='outer', on='address')
# balances = balances[(balances['value'].notna())]


# # Cut each row into their respective bins
# cut_bins = [0, 200, 400, 600, 800, 1000,
#             2000, 4000, 6000, 8000, 10000,
#             20000, 40000, 60000, 80000, 100000,
#             200000, 400000, 600000, 800000, 1000000,
#             2000000, 4000000, 6000000, 8000000, 10000000]

# balances['range'] = pd.cut(balances['value'], bins=cut_bins, right=False).astype(str)

# # df = balances.groupby('range').range.agg('count').reset_index(name='count')
# # df

# fig = px.histogram(balances, x="range")
# fig.show()

In [ ]:
# df = pd.merge(df, ids, how='outer', on='address')
# df = df[df.value.notna()]